# **Import python libraries**

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense


# **`Load the Datset from Kaggle`**

In [2]:
!pip install kaggle

import os
import zipfile

# Set up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'spikal2000'
os.environ['KAGGLE_KEY'] = '8974f6a7db0a94d996b1579c714faa18'

# Download the dataset using Kaggle API
dataset_name = 'meowmeowmeowmeowmeow/gtsrb-german-traffic-sign'
output_dir = '/content/dataset'

os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)

!kaggle datasets download -d $dataset_name

# Unzip the downloaded dataset
with zipfile.ZipFile(os.path.join(output_dir, dataset_name.split('/')[1] + '.zip'), 'r') as zip_ref:
    zip_ref.extractall()

# Continue with the rest of the code for data preprocessing, model training, and object detection


 99% 607M/612M [00:03<00:00, 210MB/s]
100% 612M/612M [00:03<00:00, 197MB/s]


# **Initializing Datset**

In [3]:
class TrafficSignsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      image_path = self.root_dir + '/' + self.data['Path'][idx]
      image = Image.open(image_path).convert('RGB')
      label = self.data['ClassId'][idx]  # Changed from 'class' to 'ClassId'
      if self.transform:
          image = self.transform(image)
      return image, label



In [4]:
# Set the path to your CSV file and image folder
csv_file_train = '/content/dataset/Train.csv'
csv_file_test = '/content/dataset/Test.csv'

image_folder = '/content/dataset'

In [5]:
# Define the transformation to apply to the training data
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize the images to a consistent size
    transforms.ToTensor(),  # Converts the images to PyTorch tensors
])

# Create the dataset
train_dataset = TrafficSignsDataset(csv_file=csv_file_train, root_dir=image_folder, transform=transform)

print(train_dataset)
test_dataset = TrafficSignsDataset(csv_file=csv_file_test, root_dir=image_folder, transform=transform)
print(test_dataset)

# **Data Augmentation**

In [6]:
# Define the transformation to apply to the training data
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])


In [7]:
# Create the training dataset with augmentation
train_dataset = TrafficSignsDataset(csv_file=csv_file_train, root_dir=image_folder, transform=transform_train)


In [8]:
# For testing, we don't want data augmentation, but we need to normalize it in the same way as the training data
transform_test = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [9]:
# Create the testing dataset without augmentation
test_dataset = TrafficSignsDataset(csv_file=csv_file_test, root_dir=image_folder, transform=transform_test)


# **Define the CNN model**

In [10]:
# Create a data loader to efficiently load the data during training
batch_size = 80
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_classes = 43


In [11]:
# Define your CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.25)


        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        self.fc1 = nn.Linear(64 * 16 * 16, 512)  # Adjusted dimensions
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.25)

        self.dropout3 = nn.Dropout(0.5)  # Additional dropout layer


        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool1(out)
        out = self.dropout1(out)


        out = self.conv3(out)
        out = self.relu3(out)
        out = self.conv4(out)
        out = self.relu4(out)
        out = self.maxpool2(out)
        out = self.dropout2(out)  # Apply the additional dropout layer

        out = out.view(out.size(0), -1)

        out = self.fc1(out)
        out = self.relu5(out)
        out = self.dropout3(out)

        out = self.fc2(out)

        return out

In [12]:
# Create an instance of CNN model
model = CNNModel(num_classes)

In [13]:
# Define loss function
criterion = nn.CrossEntropyLoss()

In [14]:
# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Train your model
num_epochs = 55

for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}/{num_epochs}")
    # Training loop
    for images, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = correct / total
    # Print training/validation metrics for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {validation_accuracy}")


Starting epoch 1/55
Epoch 1/55, Loss: 2.4168477058410645, Validation Accuracy: 0.3446555819477435
Starting epoch 2/55
Epoch 2/55, Loss: 2.9086477756500244, Validation Accuracy: 0.43151227236737927
Starting epoch 3/55
Epoch 3/55, Loss: 1.5799264907836914, Validation Accuracy: 0.5368962787015044
Starting epoch 4/55
Epoch 4/55, Loss: 1.9327744245529175, Validation Accuracy: 0.5945368171021378
Starting epoch 5/55
Epoch 5/55, Loss: 1.8490434885025024, Validation Accuracy: 0.5954077593032462
Starting epoch 6/55
Epoch 6/55, Loss: 1.1004009246826172, Validation Accuracy: 0.6527315914489311
Starting epoch 7/55
Epoch 7/55, Loss: 1.0105395317077637, Validation Accuracy: 0.6591448931116389
Starting epoch 8/55
Epoch 8/55, Loss: 1.3721158504486084, Validation Accuracy: 0.6400633412509897
Starting epoch 9/55
Epoch 9/55, Loss: 1.6756683588027954, Validation Accuracy: 0.7001583531274743
Starting epoch 10/55
Epoch 10/55, Loss: 1.341078519821167, Validation Accuracy: 0.7246239113222486
Starting epoch 11/

In [ ]:
torch.save(model.state_dict(), 'trafic_signs_classification_2.pth')